<a href="https://colab.research.google.com/github/bhadreshpsavani/EfficientQAExperiments/blob/master/Drop__1_text_summary_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
# !pip3 install os
from os import listdir
import string
from pickle import dump,load

## Amazon Food reviews Dataset

In [ ]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 552.2 kB/s eta 0:00:00


## Imports

In [ ]:
!pip install bs4
!pip install tensorflow
!pip install keras

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=e190442ee3565d6a31ae7087aaebbbec1ae677d36d368f0e80c9974758c288b2
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from keras.layers import Embedding, LSTM, Dense, TimeDistributed, Input
from keras.models import Model
from keras.callbacks import EarlyStopping

import tensorflow as tf
import time
from os import listdir
import string
from pickle import dump,load
import pickle
from keras.callbacks import ModelCheckpoint
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
os.chdir("/content/drive/MyDrive/cnn_dailymail")

AMAZON_DATA_PATH= '/content/drive/MyDrive/cnn_dailymail/Reviews.csv'

Mounted at /content/drive


In [ ]:
AMAZON_DATA_PATH= 'Reviews.csv'

In [ ]:
class Load_amazon_data:

    def __init__(self, dir_path, seed= 0):
        """
        Initialization
        """
        self.dir_path= dir_path
        np.random.seed(seed)

    def load(self):
        """
        Reads data from the given directory path
        """
        return pd.read_csv(self.dir_path)

    def drop(self):
        """
        Drops unnecessary columns
        """

        data= self.load()

        data = data.dropna()
        data= data.iloc[:, -2:]
        data = data.reset_index(drop= True)

        return data

    def analyze_data(self):
        """
        Prints some sample data points from the cleaned data
        """
        data= self.drop()

        for sr_no, i in enumerate(np.random.randint(10, 100, size= 5)):
            print("_________________________")
            print("Data Point {0}".format(sr_no + 1))
            print("Summary:")
            print(data['Summary'].iloc[i])
            print("Full Text:")
            print(data['Text'].iloc[i])

In [ ]:
obj= Load_amazon_data(AMAZON_DATA_PATH, seed= 1)

## Load the Data

In [ ]:
data= obj.load()
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labr...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as ""Jumbo""."
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all","This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with ..."
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The fl...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,"Great taffy at a great price. There was a wide assortment of yummy taffy. Delivery was very quick. If your a taffy lover, this is a deal."


## Dropping Unnecessary columns

In [ ]:
data= obj.drop()
data.head()

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labr...
1,Not as Advertised,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as ""Jumbo""."
2,"""Delight"" says it all","This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with ..."
3,Cough Medicine,If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The fl...
4,Great taffy,"Great taffy at a great price. There was a wide assortment of yummy taffy. Delivery was very quick. If your a taffy lover, this is a deal."


In [ ]:
data.shape

(568411, 2)

## Analyze the data

In [ ]:
obj.analyze_data()

_________________________
Data Point 1
Summary:
Mushy
Full Text:
The flavors are good.  However, I do not see any differce between this and Oaker Oats brand - they are both mushy.
_________________________
Data Point 2
Summary:
Delicious product!
Full Text:
I can remember buying this candy as a kid and the quality hasn't dropped in all these years. Still a superb product you won't be disappointed with.
_________________________
Data Point 3
Summary:
Forget Molecular Gastronomy - this stuff rockes a coffee creamer!
Full Text:
I know the product title says Molecular Gastronomy, but don't let that scare you off.  I have been looking for this for a while now, not for food science, but for something more down to earth.  I use it to make my own coffee creamer.<br /><br />I have to have my coffee blonde and sweet - but the flavored creamers are full of the bad kinds of fat, and honestly, I hate to use manufactured "food" items.  I really don't think they are good for the body.  On the other h

In [ ]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
class Data_cleaning:
    def __init__(self):
        self.clean_summaries= []
        self.clean_texts= []

    def clean_text(self, text, remove_stopwords = False):
        """
        Defines a series of cleaning operations
        """
        text = text.lower()

        if True:
            text = text.split()
            new_text = []
            for word in text:
                if word in contractions:
                    new_text.append(contractions[word])
                else:
                    new_text.append(word)
            text = " ".join(new_text)

        text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
        text = re.sub(r'\<a href', ' ', text)
        text = re.sub(r'&amp;', '', text)
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'<br >', ' ', text)
        text = re.sub(r'<br  >', ' ', text)
        text = re.sub(r'\'', ' ', text)

        # Optionally, remove stop words
        if remove_stopwords:
            text = text.split()
            stops = set(stopwords.words("english"))
            text = [w for w in text if not w in stops]
            text = " ".join(text)

        return text

    def clean(self, data):
        """
        Applies the clean_text() to the entire dataset
        """
        for summary in data.Summary:
            self.clean_summaries.append(self.clean_text(summary))

        print("Summaries are complete.")

        for text in data.Text:
            self.clean_texts.append(self.clean_text(text))

        print("Texts are complete.")

        return self.clean_summaries, self.clean_texts

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# import nltk
# nltk.download('stopwords')

clean_obj= Data_cleaning()
clean_summaries, clean_texts= clean_obj.clean(data)

Summaries are complete.
Texts are complete.


In [ ]:
np.random.seed(1)

for sr_no, i in enumerate(np.random.randint(10, 100, size= 5)):
    print("_________________________")
    print("Data Point #{0}".format(sr_no + 1))
    print("Summary:")
    print(clean_summaries[i])
    print("Full Text:")
    print(clean_texts[i])

_________________________
Data Point #1
Summary:
mushy
Full Text:
the flavors are good  however  i do not see any differce between this and oaker oats brand   they are both mushy 
_________________________
Data Point #2
Summary:
delicious product 
Full Text:
i can remember buying this candy as a kid and the quality has not dropped in all these years  still a superb product you will not be disappointed with 
_________________________
Data Point #3
Summary:
forget molecular gastronomy   this stuff rockes a coffee creamer 
Full Text:
i know the product title says molecular gastronomy  but do not let that scare you off  i have been looking for this for a while now  not for food science  but for something more down to earth  i use it to make my own coffee creamer   i have to have my coffee blonde and sweet   but the flavored creamers are full of the bad kinds of fat  and honestly  i hate to use manufactured  food  items  i really do not think they are good for the body  on the other hand  i

In [ ]:
#Appending new data
amazon_data= list()
for (summ, story) in zip(clean_summaries, clean_texts):
    amazon_data.append({'story': [story], 'highlights':[summ]})

In [ ]:

for sr_no, i in enumerate(np.random.randint(10, 10000, size= 5)):
    print("_________________________")
    print("Data Point #{0}".format(sr_no + 1))
    print("Summary:")
    print(amazon_data[i]['highlights'])
    print("Full Text:")
    print(amazon_data[i]['story'])

_________________________
Data Point #1
Summary:
['celebration s hibiscus tea']
Full Text:
['in an effort to reverse augmenting high blood pressure  a physician recommended i drink hibiscus herbal tea  kudos to celebration herbals for delivering the tea one day after the order was placed  i followed instructions on the box and used spring water which was steeped for approximately ten minutes  although the primary reason for drinking the tea was medicinal  i also wanted a flavorable tea  sadly the taste of the tea was unexceptional  accordingly  the next box of hibiscus tea that i purchase will not be from celebration herbals ']
_________________________
Data Point #2
Summary:
['yummy']
Full Text:
['when you have a craving for a sweet but you do not will not too much  this bar is just perfect  it is yummy ']
_________________________
Data Point #3
Summary:
['taste good but not as good as i had hoped']
Full Text:
['if you love vegetable soup you should like this  yes it has come chicken 

In [ ]:
dump(amazon_data, open('full_amazon_data.pkl', 'wb'))

In [ ]:
amazon_data= load(open('full_amazon_data.pkl', 'rb'))
amazon_data[0]

{'story': ['i have bought several of the vitality canned dog food products and have found them all to be of good quality  the product looks more like a stew than a processed meat and it smells better  my labrador is finicky and she appreciates this product better than most '],
 'highlights': ['good quality dog food']}

In [ ]:
amazon_data[5]

{'story': ['i got a wild hair for taffy and ordered this five pound bag  the taffy was all very enjoyable with many flavors  watermelon  root beer  melon  peppermint  grape  etc  my only complaint is there was a bit too much red black licorice flavored pieces  just not my particular favorites   between me  my kids  and my husband  this lasted only two weeks  i would recommend this brand of taffy    it was a delightful treat '],
 'highlights': ['nice taffy']}

In [ ]:
len(amazon_data)

568411

___

---

In [ ]:
amazon_data= amazon_data[:10000]# comment to run complete data

___

In [ ]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for word in text.split():
        if word not in count_dict:
            count_dict[word] = 1
        else:
            count_dict[word] += 1

In [ ]:
word_counts = {}
for data_point in amazon_data:
    count_words(word_counts, data_point['highlights'][0])
    count_words(word_counts, data_point['story'][0])

print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 18927


In [ ]:
print(word_counts)

{'good': 4446, 'quality': 688, 'dog': 904, 'food': 1909, 'i': 28674, 'have': 7278, 'bought': 876, 'several': 351, 'of': 13560, 'the': 31951, 'vitality': 3, 'canned': 177, 'products': 539, 'and': 22080, 'found': 927, 'them': 3364, 'all': 2832, 'to': 16878, 'be': 3163, 'product': 2847, 'looks': 144, 'more': 2279, 'like': 4454, 'a': 20424, 'stew': 16, 'than': 2059, 'processed': 60, 'meat': 191, 'it': 18527, 'smells': 133, 'better': 1420, 'my': 7772, 'labrador': 2, 'is': 15945, 'finicky': 25, 'she': 990, 'appreciates': 3, 'this': 11809, 'most': 706, 'not': 10663, 'as': 4828, 'advertised': 68, 'arrived': 301, 'labeled': 23, 'jumbo': 15, 'salted': 30, 'peanuts': 65, 'were': 1516, 'actually': 412, 'small': 571, 'sized': 73, 'unsalted': 16, 'sure': 548, 'if': 2901, 'was': 5658, 'an': 1536, 'error': 15, 'or': 3000, 'vendor': 44, 'intended': 16, 'represent': 2, 'delight': 40, 'says': 217, 'confection': 7, 'that': 7299, 'has': 2241, 'been': 1187, 'around': 363, 'few': 609, 'centuries': 1, 'light'

In [ ]:
import numpy as np

#We are reading a file (numberbatch-en.txt) line by line, where each line contains a word followed by its embedding values.
#It splits each line into values, extracts the word and its corresponding embedding, and stores them in the embeddings_index dictionary.
embeddings_index = {}
with open('/content/drive/MyDrive/cnn_dailymail/amazon/numberbatch-en.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.array(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 516783


In [ ]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1

missing_ratio = round(missing_words/len(word_counts),5)*100

print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {0}%".format(missing_ratio))

Number of words missing from CN: 57
Percent of words that are missing from vocabulary: 0.301%


In [ ]:
# with open("/home/nikhil/Music/Text-Summarization/Embeddings/wiki-news-300d-1M.vec") as f:
#     count= 0
#     for line in f:
#         vals= line.split()
#         word= vals[0]
#         vector= vals[1:]
#         count= count+1
#         print(len(vector))
#         if count==10:
#             break
#         print(word)



In [ ]:
word_counts

{'good': 4446,
 'quality': 688,
 'dog': 904,
 'food': 1909,
 'i': 28674,
 'have': 7278,
 'bought': 876,
 'several': 351,
 'of': 13560,
 'the': 31951,
 'vitality': 3,
 'canned': 177,
 'products': 539,
 'and': 22080,
 'found': 927,
 'them': 3364,
 'all': 2832,
 'to': 16878,
 'be': 3163,
 'product': 2847,
 'looks': 144,
 'more': 2279,
 'like': 4454,
 'a': 20424,
 'stew': 16,
 'than': 2059,
 'processed': 60,
 'meat': 191,
 'it': 18527,
 'smells': 133,
 'better': 1420,
 'my': 7772,
 'labrador': 2,
 'is': 15945,
 'finicky': 25,
 'she': 990,
 'appreciates': 3,
 'this': 11809,
 'most': 706,
 'not': 10663,
 'as': 4828,
 'advertised': 68,
 'arrived': 301,
 'labeled': 23,
 'jumbo': 15,
 'salted': 30,
 'peanuts': 65,
 'were': 1516,
 'actually': 412,
 'small': 571,
 'sized': 73,
 'unsalted': 16,
 'sure': 548,
 'if': 2901,
 'was': 5658,
 'an': 1536,
 'error': 15,
 'or': 3000,
 'vendor': 44,
 'intended': 16,
 'represent': 2,
 'delight': 40,
 'says': 217,
 'confection': 7,
 'that': 7299,
 'has': 2241,

In [ ]:
#dictionary to convert words to integers
vocab_to_int = {}

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index: #evevn if the count is less than threshold, we have its embedding
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 18927
Number of words we will use: 15749
Percent of words we will use: 83.21%


In [ ]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

15749


In [ ]:
word_embedding_matrix.shape

(15749, 300)

In [ ]:
def convert_to_ints(sentence, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''

    sentence_ints = []
    for word in sentence.split():
        word_count += 1
        if word in vocab_to_int:
            sentence_ints.append(vocab_to_int[word])
        else:
            sentence_ints.append(vocab_to_int["<UNK>"])
            unk_count += 1
    if eos:
        sentence_ints.append(vocab_to_int["<EOS>"])
    return sentence_ints, word_count, unk_count

In [ ]:
word_count = 0
unk_count = 0

int_summaries= list()
int_texts= list()

for data_point in amazon_data:
    summaries, word_count, unk_count = convert_to_ints(data_point['highlights'][0], word_count, unk_count)
    texts, word_count, unk_count = convert_to_ints(data_point['story'][0], word_count, unk_count, eos=True)
    int_summaries.append(summaries)
    int_texts.append(texts)


unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 802200
Total number of UNKs in headlines: 5749
Percent of words that are UNK: 0.72%


In [ ]:
import pandas as pd
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [ ]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
             counts
count  10000.000000
mean       4.138000
std        2.658508
min        0.000000
25%        2.000000
50%        4.000000
75%        5.000000
max       30.000000

Texts:
             counts
count  10000.000000
mean      77.082000
std       71.997943
min        1.000000
25%       34.000000
50%       56.000000
75%       95.000000
max     1419.000000


In [ ]:
print(lengths_summaries.shape)
# lengths_summaries

(10000, 1)


In [ ]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

150.0
200.0
353.0100000000002


In [ ]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

8.0
9.0
13.0


In [ ]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [ ]:
min(lengths_texts.counts)

1

In [ ]:
# takes a long time  , this is normal

# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 158
max_summary_length = 8
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length):
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])  #SO that points are not repeated
           ):

            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])

# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

6701
6701


In [ ]:
len(vocab_to_int)

15749

In [ ]:

dump(sorted_texts, open('sorted_texts.pkl', 'wb'))
dump(sorted_summaries, open('sorted_summaries.pkl', 'wb'))

In [ ]:
dump(word_embedding_matrix, open('word_embedding_matrix.pkl', 'wb'))
dump(vocab_to_int, open('vocab_to_int.pkl', 'wb'))
dump(int_to_vocab, open('int_to_vocab.pkl', 'wb'))


In [ ]:
from pickle import dump,load
sorted_texts= load(open('sorted_texts.pkl', 'rb'))
sorted_summaries= load(open('sorted_summaries.pkl', 'rb'))

In [ ]:
word_embedding_matrix= load(open('word_embedding_matrix.pkl', 'rb'))
vocab_to_int= load(open('vocab_to_int.pkl', 'rb'))
int_to_vocab= load(open('int_to_vocab.pkl', 'rb'))


In [ ]:
# sample_text= sorted_texts[50]
# sample_summary= sorted_summaries[105]

In [ ]:
# sample_text= sorted_texts[100]
# sample_summary= sorted_summaries[105]

# sample_sent_text= list()
# for word in sample_text:
#     sample_sent_text.append(int_to_vocab[word])


In [ ]:
# sample_sent_text

In [ ]:
# sample_sent_summ= list()
# for word in sample_summary:
#     sample_sent_summ.append(int_to_vocab[word])

In [ ]:
# sample_sent_summ

## Text Padding

In [ ]:
max_text_length= 158
max_summary_length= 8 + 1

def transform_input_text(texts):
    padded_texts= list()
    for text in texts:
        if len(text) < max_text_length:
            text= text + [vocab_to_int['<PAD>'] for i in range(max_text_length - len(text))]

        else:
            text= text[:max_text_length]
        padded_texts.append(text)

    return padded_texts

# def transform_output_text(texts):
#     padded_texts= list()
#     max_length_texts= list()

#     for text in texts:
#         text= text[:max_summary_length]
#         max_length_texts.append(text)

#     for text in max_length_texts:
#         text= [vocab_to_int['<GO>']] + text + [vocab_to_int['<EOS>']]
#         if len(text) < max_summary_length:
#             text= text + [vocab_to_int['<PAD>'] for i in range(max_summary_length - len(text))]

#         padded_texts.append(text)

#     return padded_texts


In [ ]:
# def pad_and_start_symbs(text):

#     text= [vocab_to_int['<GO>']] + text + [vocab_to_int['<EOS>']]
#     if len(text) < max_summary_length:
#         text= text + [vocab_to_int['<PAD>'] for i in range(max_summary_length - len(text))]


In [ ]:
padded_sorted_texts= transform_input_text(sorted_texts)
padded_sorted_summaries= sorted_summaries

# padded_sorted_summaries= transform_output_text(sorted_summaries)

In [ ]:
# padded_sorted_summaries= padded_sorted_summaries[-15000:]
# padded_sorted_texts= padded_sorted_texts[-15000:]

In [ ]:
len(padded_sorted_texts)

6701

In [ ]:
word_embedding_matrix.shape

(15749, 300)

## Model Building
The encoder-decoder model for recurrent neural networks is an architecture for sequence-to-sequence prediction problems.

It is comprised of two sub-models, as its name suggests:

    Encoder: The encoder is responsible for stepping through the input time steps and encoding the entire sequence into a fixed length vector called a context vector.
    Decoder: The decoder is responsible for stepping through the output time steps while reading from the context vector.


## Real-time Data Generation for Keras model

In [ ]:
def pad_text(text):
    if len(text) < max_summary_length:
        text= text + [vocab_to_int['<PAD>'] for i in range(max_summary_length - len(text))]

    return text

In [ ]:
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates batches of data for keras'
    def __init__(self, texts, summaries, word_embedding_matrix, batch_size= 64,
                 n_classes= len(vocab_to_int), shuffle= True, EMBEDDING_SIZE= 300, mode= 'train'):
        'Initialization'

        self.batch_size = batch_size
        self.max_text_length= max_text_length

        self.max_summ_length= max_summary_length
        self.vocab_to_int= vocab_to_int
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.mode= mode

        brk_point= int(len(texts)*0.8)
        if self.mode == 'train':
            texts= texts[:brk_point]
            summaries= summaries[:brk_point]
        else:
            texts= texts[brk_point:]
            summaries= summaries[brk_point:]

        self.texts= texts
        self.summaries= summaries

        self.word_embedding_matrix= word_embedding_matrix
        self.EMBEDDING_SIZE= EMBEDDING_SIZE

        self.on_epoch_end()

    def __len__(self):   #Correct
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.texts) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        [X, Z], y = self.__data_generation(indexes)
        return [X, Z], y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.texts))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, self.max_text_length, self.EMBEDDING_SIZE))
        y = np.empty((self.batch_size, self.max_summ_length, self.n_classes), dtype=int)
        Z = np.empty((self.batch_size, self.max_summ_length, self.EMBEDDING_SIZE))

        # Generate data
        for i, ID in enumerate(indexes):

            text_data_point= self.texts[ID]
            summary_data_point= self.summaries[ID]

            for j, word in enumerate(text_data_point):
                emb= self.word_embedding_matrix[word]
                X[i, j, ]= emb

            input_summary_point= [self.vocab_to_int["<GO>"]] + summary_data_point
            output_summary_point= summary_data_point

            input_summary_point= input_summary_point[:max_summary_length]
            output_summary_point= output_summary_point[:max_summary_length -1]

            output_summary_point= output_summary_point + [self.vocab_to_int['<EOS>']]


            input_summary_point= pad_text(input_summary_point)
            output_summary_point= pad_text(output_summary_point)

            for j, word in enumerate(input_summary_point):

                emb= self.word_embedding_matrix[word]
                Z[i, j, ]= emb

            for j, word in enumerate(output_summary_point):
                temp= np.zeros((self.n_classes, ))
                temp[word]= 1
                y[i, j, ]= temp

        return [X, Z], y

In [ ]:
import numpy as np
train_gen= DataGenerator(padded_sorted_texts, padded_sorted_summaries, word_embedding_matrix, mode= 'train')

val_gen= DataGenerator(padded_sorted_texts, padded_sorted_summaries, word_embedding_matrix, mode= 'val')

In [ ]:
train_gen

## Buffer Implementation

In [ ]:
# from keras.utils import to_categorical
# import numpy as np
# def generate_batch(self, sorted_texts, sorted_summaries, batch_size= 100):
#     num_batches = len(x_samples) // batch_size
#     while True:
#         for batchIdx in range(0, num_batches):
#             start = batchIdx * batch_size
#             end = (batchIdx + 1) * batch_size
#             encoder_input_data_batch = pad_sequences(x_samples[start:end], self.max_input_seq_length)
#             decoder_target_data_batch = np.zeros(shape=(batch_size, self.max_target_seq_length, self.num_target_tokens))
#             decoder_input_data_batch = np.zeros(shape=(batch_size, self.max_target_seq_length, GLOVE_EMBEDDING_SIZE))
#             for lineIdx, target_words in enumerate(y_samples[start:end]):
#                 for idx, w in enumerate(target_words):
#                     w2idx = 0  # default [UNK]
#                     if w in self.word2em:
#                         emb = self.unknown_emb
#                         decoder_input_data_batch[lineIdx, idx, :] = emb
#                     if w in self.target_word2idx:
#                         w2idx = self.target_word2idx[w]
#                     if w2idx != 0:
#                         if idx > 0:
#                             decoder_target_data_batch[lineIdx, idx - 1, w2idx] = 1
#             yield [encoder_input_data_batch, decoder_input_data_batch], decoder_target_data_batch

___

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# class Seq2seq:
#     def __init__(self, vocab_to_int, int_to_vocab, word_embedding_matrix):
#         self.max_text_length= max_text_length
#         self.max_summary_length= max_summary_length
#         self.vocab_to_int= vocab_to_int
#         self.int_to_vocab= int_to_vocab
#         self.vocab_size= len(self.vocab_to_int)
#         self.EMBEDDING_SIZE= 300
#         self.HIDDEN_UNITS= 100

#         self.model_path= '/content/drive/MyDrive/cnn_dailymail/amazon/saved_model/model.h5'
#         self.word_embedding_matrix= word_embedding_matrix
#         self.unknown_emb= self.word_embedding_matrix[self.vocab_to_int['<UNK>']]

#         encoder_inputs= Input(shape= (None, self.EMBEDDING_SIZE), name= 'encoder_inputs')
#         encoder_lstm = LSTM(units= self.HIDDEN_UNITS, return_state= True, name= 'encoder_lstm')

#         encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_inputs)
#         encoder_states = [encoder_state_h, encoder_state_c]

#         decoder_inputs = Input(shape= (None, self.EMBEDDING_SIZE), name= 'decoder_inputs')
#         decoder_lstm = LSTM(units= self.HIDDEN_UNITS, return_state=True, return_sequences=True, name='decoder_lstm')

#         decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_inputs,
#                                                                          initial_state= encoder_states)
#         decoder_dense = Dense(units= self.vocab_size, activation= 'softmax', name= 'decoder_dense')
#         decoder_outputs = decoder_dense(decoder_outputs)

#         model = Model(inputs= [encoder_inputs, decoder_inputs], outputs= decoder_outputs)

#         model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

#         self.model = model

#         self.encoder_model = Model(encoder_inputs, encoder_states)

#         decoder_state_inputs = [Input(shape= (self.HIDDEN_UNITS,)), Input(shape= (self.HIDDEN_UNITS,))]
#         decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state= decoder_state_inputs)
#         decoder_states = [state_h, state_c]
#         decoder_outputs = decoder_dense(decoder_outputs)

#         self.decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

#     def fit(self, train_gen, val_gen):
#         from keras.models import load_model
#         from keras.callbacks import ModelCheckpoint

#         checkpoint = ModelCheckpoint(self.model_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

#         self.model.fit_generator(generator= train_gen, validation_data= val_gen, epochs=200, callbacks= [checkpoint])

#         self.model.save(self.model_path)  # creates a HDF5 file 'my_model.h5'
#         self.model = load_model(self.model_path)

#         return self.model, self.encoder_model, self. decoder_model

#     def summarize(self, input_text, model, encoder_model, decoder_model):

#         self.model= model
#         self.encoder_model= encoder_model
#         self.decoder_model= decoder_model

#         input_seq = np.zeros((1, self.max_text_length, self.EMBEDDING_SIZE))
#         for idx, word in enumerate(input_text.lower().split(' ')):
#             if idx >= self.max_text_length:
#                 break
#             emb = self.unknown_emb  # default [UNK]
#             if word in self.vocab_to_int:
#                 emb = self.word_embedding_matrix[self.vocab_to_int[word]]
#             input_seq[0, idx, :] = emb

#         states_value = self.encoder_model.predict(input_seq)
#         target_seq = np.zeros((1, 1, self.EMBEDDING_SIZE))
#         target_seq[0, 0, :] = self.word_embedding_matrix[self.vocab_to_int['<GO>']]
#         target_text = ''
#         target_text_len = 0
#         terminated = False
#         while not terminated:
#             output_tokens, h, c = self.decoder_model.predict([target_seq] + states_value)

#             sample_token_idx = np.argmax(output_tokens[0, -1, :])
#             sample_word = self.int_to_vocab[sample_token_idx]
#             target_text_len += 1

# #             if sample_word != '<GO>' and sample_word != '<EOS>':
#             target_text += ' ' + sample_word

#             if sample_word == '<EOS>' or target_text_len >= self.max_summary_length:
#                 terminated = True

#             if sample_word in self.vocab_to_int:
#                 target_seq[0, 0, :] = self.word_embedding_matrix[self.vocab_to_int[sample_word]]

#             else:
#                 target_seq[0, 0, :] = self.unknown_emb

#             states_value = [h, c]
#         return target_text.strip()

In [ ]:
from keras import backend as K

def cosine_similarity(y_true, y_pred):
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    return K.sum(y_true * y_pred, axis=-1)

In [ ]:
max_text_length= max_text_length
max_summary_length= max_summary_length
vocab_to_int= vocab_to_int
int_to_vocab= int_to_vocab
vocab_size= len(vocab_to_int)
EMBEDDING_SIZE= 300
HIDDEN_UNITS= 100

model_path= 'model.h5'
#         self.word_embedding_matrix= word_embedding_matrix
unknown_emb= word_embedding_matrix[vocab_to_int['<UNK>']]

encoder_inputs= Input(shape= (None, EMBEDDING_SIZE), name= 'encoder_inputs')
encoder_lstm = LSTM(units= HIDDEN_UNITS, return_state= True, name= 'encoder_lstm')

encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_inputs)
encoder_states = [encoder_state_h, encoder_state_c]

decoder_inputs = Input(shape= (None, EMBEDDING_SIZE), name= 'decoder_inputs')
decoder_lstm = LSTM(units= HIDDEN_UNITS, return_state=True, return_sequences=True, name='decoder_lstm')

decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_inputs,
                                                                 initial_state= encoder_states)
decoder_dense = Dense(units= vocab_size, activation= 'softmax', name= 'decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model(inputs= [encoder_inputs, decoder_inputs], outputs= decoder_outputs)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=[cosine_similarity])

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_inputs = [Input(shape= (HIDDEN_UNITS,)), Input(shape= (HIDDEN_UNITS,))]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state= decoder_state_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)


from keras.models import load_model
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

model.fit_generator(generator= train_gen, validation_data= val_gen, epochs=200, callbacks= [checkpoint])

model.save(model_path)  # creates a HDF5 file 'my_model.h5'
model = load_model(model_path)


Epoch 1/200
83/83 [==============================] - ETA: 0s - loss: 5.5790 - cosine_similarity: 0.3930
Epoch 1: val_loss improved from inf to 4.03931, saving model to model.h5
83/83 [==============================] - 17s 172ms/step - loss: 5.5790 - cosine_similarity: 0.3930 - val_loss: 4.0393 - val_cosine_similarity: 0.4794
Epoch 2/200
83/83 [==============================] - ETA: 0s - loss: 3.1933 - cosine_similarity: 0.5685
Epoch 2: val_loss improved from 4.03931 to 3.69190, saving model to model.h5
83/83 [==============================] - 14s 162ms/step - loss: 3.1933 - cosine_similarity: 0.5685 - val_loss: 3.6919 - val_cosine_similarity: 0.5170
Epoch 3/200
83/83 [==============================] - ETA: 0s - loss: 2.9583 - cosine_similarity: 0.6033
Epoch 3: val_loss improved from 3.69190 to 3.58915, saving model to model.h5
83/83 [==============================] - 14s 164ms/step - loss: 2.9583 - cosine_similarity: 0.6033 - val_loss: 3.5891 - val_cosine_similarity: 0.5272
Epoch 4/200

In [ ]:
# obj= Seq2seq(vocab_to_int, int_to_vocab, word_embedding_matrix)
# model, encoder, decoder= obj.fit(train_gen, val_gen)

In [ ]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None, 300)]          0         []                            
 )                                                                                                
                                                                                                  
 decoder_inputs (InputLayer  [(None, None, 300)]          0         []                            
 )                                                                                                
                                                                                                  
 encoder_lstm (LSTM)         [(None, 100),                160400    ['encoder_inputs[0][0]']      
                              (None, 100),                                                  

In [ ]:
decoder_model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 decoder_inputs (InputLayer  [(None, None, 300)]          0         []                            
 )                                                                                                
                                                                                                  
 input_8 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 input_9 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 decoder_lstm (LSTM)         [(None, None, 100),          160400    ['decoder_inputs[0][0]',

# **Prediction**

In [ ]:
sample= ['first',
 'time',
 'buying',
 'this',
 'and',
 'will',
 'not',
 'buy',
 'it',
 'again',
 'i',
 'have',
 'tried',
 'the',
 'low',
 'fat',
 'silverado',
 'chili',
 'and',
 'it',
 'is',
 'just',
 'as',
 'mild',
 'as',
 'this',
 'one',
 'difference',
 'is',
 'it',
 'is',
 'not',
 'loaded',
 'with',
 'unnecessary',
 'lard',
 'and',
 'i',
 'can',
 'add',
 'my',
 'own',
 'heat',
 'this',
 'one',
 'is',
 'not',
 'the',
 'least',
 'bit',
 'hot',
 'if',
 'a',
 'company',
 'is',
 'going',
 'to',
 'market',
 'something',
 'as',
 'ignited',
 'by',
 'habanero',
 'it',
 'better',
 'have','habanero',
 'in',
 'it',
 'i',
 'have',
 'had',
 'campbell',
 's',
 'firehouse',
 'chili',
 'and',
 'that',
 'one',
 'is',
 'a',
 'little',
 'bit',
 'spicier',
 'forget',
 'has',
 'less',
 'calories',
 'still',
 'prefer',
 'the','high','fat','silverado','though', 'i','ll','be','giving','away','the','rest','of','this','6', 'pack', '<EOS>']


In [ ]:
EMBEDDING_SIZE= 300

def summarize(input_text, model, encoder_model, decoder_model):
    """
    Sumarize
    """
    input_seq = np.zeros((1, max_text_length, EMBEDDING_SIZE))
    for idx, word in enumerate(input_text.lower().split(' ')):
        if idx >= max_text_length:
            break
        emb = word_embedding_matrix[vocab_to_int['<UNK>']]  # default [UNK]
        if word in vocab_to_int:
            emb = word_embedding_matrix[vocab_to_int[word]]
        input_seq[0, idx, :] = emb

    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, EMBEDDING_SIZE))
    target_seq[0, 0, :] = word_embedding_matrix[vocab_to_int['<GO>']]
    target_text = ''
    target_text_len = 0
    terminated = False
    while not terminated:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sample_token_idx = np.argmax(output_tokens[0, -1, :])
        sample_word = int_to_vocab[sample_token_idx]
        target_text_len += 1

#         if sample_word != '<GO>' and sample_word != '<EOS>':
        target_text += ' ' + sample_word

        if sample_word == '<EOS>' or target_text_len >= max_summary_length:
            terminated = True

        if sample_word in vocab_to_int:
            target_seq[0, 0, :] = word_embedding_matrix[vocab_to_int[sample_word]]

        else:
            target_seq[0, 0, :] = word_embedding_matrix[vocab_to_int['<UNK>']]

        states_value = [h, c]
    return target_text.strip()

In [ ]:
sample_text= sorted_texts[3255]
#sample_summary= sorted_summaries[1]

sample_sent_text= list()
for word in sample_text:
    sample_sent_text.append(int_to_vocab[word])

sent=""
for i in range(len(sample_sent_text)-1):
    sent+=sample_sent_text[i] + " "

sent

'ordered 2 cases on different dates and got them within 2 days cant find these in the stores so please keep them in stock make for a great meal with some veggies or a salad and this type of fish with omega 3 s is good for you '

In [ ]:
summarize(sent, model, encoder_model, decoder_model)

1/1 [==============================] - 0s 27ms/step


'great <EOS>'

In [ ]:
sample_text3= sorted_texts[5]
sample_summary3= sorted_summaries[105]

sample_sent_text3= list()
for word in sample_text3:
    sample_sent_text3.append(int_to_vocab[word])

sent3=""
for i in range(len(sample_sent_text3)-1):
    sent3+=sample_sent_text3[i] + " "

sent3


'product received is as advertised '

In [ ]:
summarize(sent3, model, encoder_model, decoder_model)

1/1 [==============================] - 0s 18ms/step


'great <EOS>'

In [ ]:

from keras.layers import RepeatVector

vocab_size = len(vocab_to_int)
src_txt_length = max_text_length
sum_txt_length = max_summary_length
# encoder input model
inputs = Input(shape=(src_txt_length,))
encoder1 = Embedding(vocab_size, 128)(inputs)
encoder2 = LSTM(128)(encoder1)
encoder3 = RepeatVector(sum_txt_length)(encoder2)
# decoder output model
decoder1 = LSTM(128, return_sequences=True)(encoder3)
outputs = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder1)
# tie it together
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

## Buffer Implementation

In [ ]:
def define_models(n_input= 300, d_input= 300, n_output= len(vocab_to_int), n_units= 150):
	# define training encoder
	encoder_inputs = Input(shape= (None, n_input))
	encoder = LSTM(n_units, return_state= True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	# define training decoder
	decoder_inputs = Input(shape= (None, d_input))
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state= encoder_states)
	decoder_dense = Dense(n_output, activation='softmax')
	decoder_outputs = decoder_dense(decoder_outputs)
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))

	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model

In [ ]:
model, encoder_model, decoder_model= define_models()

In [ ]:
#keras.utils.print_summary(decoder_model)

In [ ]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, None, 300)]          0         []                            
                                                                                                  
 input_12 (InputLayer)       [(None, None, 300)]          0         []                            
                                                                                                  
 lstm_6 (LSTM)               [(None, 150),                270600    ['input_11[0][0]']            
                              (None, 150),                                                        
                              (None, 150)]                                                        
                                                                                           

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[cosine_similarity])

In [ ]:
import numpy as np

In [ ]:
gen= DataGenerator(padded_sorted_texts, padded_sorted_summaries, word_embedding_matrix)

In [ ]:
model.fit_generator(generator= gen, epochs= 2)

Epoch 1/2
83/83 [==============================] - 19s 181ms/step - loss: 4.6904 - cosine_similarity: 0.4158
Epoch 2/2
83/83 [==============================] - 12s 145ms/step - loss: 2.9667 - cosine_similarity: 0.5876


## Prediction

In [ ]:
sample_text3= sorted_texts[100]
sample_summary3= sorted_summaries[105]

sample_sent_text3= list()
for word in sample_text3:
    sample_sent_text3.append(int_to_vocab[word])

sent3=""
for i in range(len(sample_sent_text3)-1):
    sent3+=sample_sent_text3[i] + " "

sent3

'this coffee is a smooth yet robust taste that does not have any bad after taste '

In [ ]:
summarize(sent3, model, encoder_model, decoder_model)

1/1 [==============================] - 0s 18ms/step


'great <EOS>'